# Milestone 3 : Predicting the next summer hit !
__Write random BS about the project__

## Must have

- Extract and match billboard rankings __Parsing_test__
- Extract artist tags and  __DONE in tags__
- Extract popular pitches
- Find language based on lyrics (words)
- Most frequent words
- Artist coords (if relevent)

- Find relation between the songs/success/features by year 
- ML on the features

- Do da report

## Nice to have 
- Sentiment analysis on lyrics and find keywords for summer hit
- Match sentiment with song feature (Sad lyrics == minor mode ? )
